# Instructions for Wrangling Match Info

This file was made to streamline the acquisition process. You can make a copy of this file or just copy all of the code into a new one to run it.

In [1]:
import numpy as np
import pandas as pd
import requests
import prepare
from time import sleep
from env import api_key

In [2]:
#Load the match_ids.csv file
matches_df = pd.read_csv('match_ids.csv')
matches_df

,0
0,NA1_4062082172
1,NA1_4093908534
2,NA1_4094915650
3,NA1_4082128299
4,NA1_4083500908
...,...
37279,NA1_4097058147
37280,NA1_4096721229
37281,NA1_4094049450
37282,NA1_4091691873


There are 37,284 match ids that need to be run. The first 4,000 have already been acquired, so I will divide the rest of them between the 5 of us. Four of us will acquire 6,657 while the last person will only acquire 6,656.

In [3]:
#Set up the match id list. You only need to uncomment the line following your name

#Johnathon
#match_ids = list(matches_df['0'])[4000:10_657]

#Joshua Bryant
#match_ids = list(matches_df['0'])[10_657:17_314]

#Jared
#match_ids = list(matches_df['0'])[17_314:23_971]

#Chris
match_ids = list(matches_df['0'])[23_971:25_628]

#Joshua Chavez
#match_ids = list(matches_df['0'])[30_628:37_284]

In [4]:
#Set up your username. You only need to uncomment the line following your name.
#Again, this username is only for naming the json files.

#Johnathon
#username = 'smith'

#Joshua Bryant
#username = 'bryant'

#Jared
#username = 'vahle'

#Chris
username = 'everts'

#Joshua Chavez
#username = 'chavez'

### Define the Function

In [5]:
def get_match_info(match_ids, api_key, username, time = 20):
    """
    This function takes in a list of match ids and iterates through them. For each iteration,
    it will make two api calls and retrieve the necessary information for our project.
    
    Two json lists will be created, and once all the info has been gathered for each match id,
    this function will save both json lists for future reference. 
    
    This function also takes in a username string. This will be used for naming the saved files.
    
    Finally, this function will funnel the resulting json lists into a prepare function and
    return a complete df.
    
    The time parameter is used in the prepare function that will be called at the end
    and represents the timeframe of the data we want to gather and prepare.
    The default value is 20 for the 20 minute mark of the match.
    
    """
    
    #Create an empty list to store the timeline json data
    timeline_data = []
    
    #Create an empty list to store the other json data
    game_data = []
    
    #Set up main url
    url = 'https://americas.api.riotgames.com/'
    
    #Create a count var
    n = 0
    
    #Store the length of the match_ids list in a var
    match_count = len(match_ids)
    
    #Loop through each match_id
    for match in match_ids:
        #Update the count var
        n += 1
        
        #After every 50 match ids (100 total requests), wait 150 seconds (2.5 minutes)
        if n % 50 == 0:
            print(f'Completed {n} of {match_count} match IDs.')
            print(f'Waiting 150 seconds...\n')
            
            #Sleep 150 seconds
            sleep(150)
            
            print('Continuing!\n')
        
        ############################# TIMELINE DATA #######################
        #Set up timeline url
        timeline_query = f'lol/match/v5/matches/{match}/timeline/?api_key={api_key}'
        
        #Grab timeline json data
        timeline_response = requests.get(url + timeline_query)
        
        #Check response and leave a status message
        if timeline_response.status_code != 200:
            print(f'Something went wrong getting TIMELINE DATA! Status Code {timeline_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID.\n')
            continue

        #Turn it into json format
        timeline_json = timeline_response.json()
        
        #Append this data to the timeline_data list
        timeline_data.append(timeline_json)
        
        ############################# OTHER GAME DATA #######################
        #Set up game data url
        game_query = f'lol/match/v5/matches/{match}?api_key={api_key}'
        
        #Grab game json data
        game_response = requests.get(url + game_query)
        
        #Check response and leave a status message.
        if game_response.status_code != 200:
            print(f'Something went wrong getting OTHER GAME DATA! Status Code {game_response.status_code} for match ID: {match}.')
            print(f'Skipping this match ID and REMOVING PREVIOUS TIMELINE ENTRY.\n')
            
            #Remove the last entry in the timeline_data list
            timeline_data.pop()
            continue
        
        #Turn it into json format
        game_json = game_response.json()
        
        #Append this data to the game_data list
        game_data.append(game_json)
        
    ################################## END LOOP ##############################
    
    #Now that all of the json data has been gathered, save each file for future reference
    
    #Save timeline data.
    #First, convert it to a df
    timeline_df = pd.DataFrame(timeline_data)
    
    #Save as csv file. Use username string to identify whose file it is
    timeline_df.to_json(f'timeline_data_{username}.json')
    
    #Print status message
    print(f'Created timeline_data_{username}.json')
    
    #Save other game data
    #First, convert it to a df
    game_df = pd.DataFrame(game_data)
    
    #Save as csv file. Use username string to identify whose file it is
    game_df.to_json(f'other_game_data_{username}.json')
    
    #Print status message
    print(f'Created other_game_data_{username}.json\n')
    
    ################################ PREPARE DATA #########################
    
    #The following section will funnel the data into the prepare function
    #Written by Joshua C.
    
    #Leave a status message
    print('Preparing the data...\n')
    
    #Begin preparing the data
    df = prepare.prepare(timeline_data, game_data, time)
    
    #Finally, return the prepared df
    return df

### Use the Function

Once the function is finished running, you will have two new json files. One for the timeline data, and another for the other game data. This function also returns a prepared df that you can immediately save as a .csv if you like.

In [6]:
#Call the get_match_info function
df = get_match_info(match_ids, api_key, username)

Completed 50 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 100 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 150 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 200 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 250 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 300 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 350 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 400 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 450 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 500 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 550 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 600 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 650 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 700 of 1657 match IDs.
Waiting 150 seconds...

Continuing!

Completed 750 of 1657

Finished with: 175 of 1657
Finished with: 176 of 1657
Skipping: 177 due to <20 min or not classic
Skipping: 178 due to <20 min or not classic
Skipping: 179 due to <20 min or not classic
Skipping: 180 due to <20 min or not classic
Skipping: 181 due to <20 min or not classic
Skipping: 182 due to <20 min or not classic
Finished with: 183 of 1657
Finished with: 184 of 1657
Finished with: 185 of 1657
Finished with: 186 of 1657
Finished with: 187 of 1657
Skipping: 188 due to <20 min or not classic
Finished with: 189 of 1657
Finished with: 190 of 1657
Finished with: 191 of 1657
Finished with: 192 of 1657
Finished with: 193 of 1657
Finished with: 194 of 1657
Finished with: 195 of 1657
Finished with: 196 of 1657
Skipping: 197 due to <20 min or not classic
Finished with: 198 of 1657
Finished with: 199 of 1657
Finished with: 200 of 1657
Finished with: 201 of 1657
Finished with: 202 of 1657
Finished with: 203 of 1657
Finished with: 204 of 1657
Finished with: 205 of 1657
Skipping: 206 due to <20 mi

Finished with: 434 of 1657
Finished with: 435 of 1657
Finished with: 436 of 1657
Finished with: 437 of 1657
Finished with: 438 of 1657
Finished with: 439 of 1657
Finished with: 440 of 1657
Finished with: 441 of 1657
Skipping: 442 due to <20 min or not classic
Finished with: 443 of 1657
Finished with: 444 of 1657
Finished with: 445 of 1657
Skipping: 446 due to <20 min or not classic
Finished with: 447 of 1657
Finished with: 448 of 1657
Finished with: 449 of 1657
Finished with: 450 of 1657
Finished with: 451 of 1657
Finished with: 452 of 1657
Finished with: 453 of 1657
Finished with: 454 of 1657
Skipping: 455 due to <20 min or not classic
Finished with: 456 of 1657
Finished with: 457 of 1657
Finished with: 458 of 1657
Finished with: 459 of 1657
Skipping: 460 due to <20 min or not classic
Finished with: 461 of 1657
Finished with: 462 of 1657
Finished with: 463 of 1657
Skipping: 464 due to <20 min or not classic
Finished with: 465 of 1657
Finished with: 466 of 1657
Finished with: 467 of 16

Finished with: 691 of 1657
Finished with: 692 of 1657
Finished with: 693 of 1657
Finished with: 694 of 1657
Skipping: 695 due to <20 min or not classic
Skipping: 696 due to <20 min or not classic
Skipping: 697 due to <20 min or not classic
Skipping: 698 due to <20 min or not classic
Finished with: 699 of 1657
Finished with: 700 of 1657
Finished with: 701 of 1657
Finished with: 702 of 1657
Skipping: 703 due to <20 min or not classic
Finished with: 704 of 1657
Finished with: 705 of 1657
Skipping: 706 due to <20 min or not classic
Skipping: 707 due to <20 min or not classic
Skipping: 708 due to <20 min or not classic
Skipping: 709 due to <20 min or not classic
Finished with: 710 of 1657
Finished with: 711 of 1657
Skipping: 712 due to <20 min or not classic
Finished with: 713 of 1657
Finished with: 714 of 1657
Skipping: 715 due to <20 min or not classic
Finished with: 716 of 1657
Skipping: 717 due to <20 min or not classic
Finished with: 718 of 1657
Finished with: 719 of 1657
Finished with

Finished with: 923 of 1657
Finished with: 924 of 1657
Finished with: 925 of 1657
Finished with: 926 of 1657
Finished with: 927 of 1657
Finished with: 928 of 1657
Finished with: 929 of 1657
Skipping: 930 due to <20 min or not classic
Skipping: 931 due to <20 min or not classic
Finished with: 932 of 1657
Finished with: 933 of 1657
Finished with: 934 of 1657
Skipping: 935 due to <20 min or not classic
Skipping: 936 due to <20 min or not classic
Finished with: 937 of 1657
Finished with: 938 of 1657
Skipping: 939 due to <20 min or not classic
Finished with: 940 of 1657
Skipping: 941 due to <20 min or not classic
Finished with: 942 of 1657
Skipping: 943 due to <20 min or not classic
Skipping: 944 due to <20 min or not classic
Finished with: 945 of 1657
Skipping: 946 due to <20 min or not classic
Skipping: 947 due to <20 min or not classic
Finished with: 948 of 1657
Finished with: 949 of 1657
Finished with: 950 of 1657
Finished with: 951 of 1657
Finished with: 952 of 1657
Skipping: 953 due to

Finished with: 1161 of 1657
Skipping: 1162 due to <20 min or not classic
Skipping: 1163 due to <20 min or not classic
Skipping: 1164 due to <20 min or not classic
Finished with: 1165 of 1657
Finished with: 1166 of 1657
Finished with: 1167 of 1657
Finished with: 1168 of 1657
Finished with: 1169 of 1657
Finished with: 1170 of 1657
Finished with: 1171 of 1657
Finished with: 1172 of 1657
Finished with: 1173 of 1657
Finished with: 1174 of 1657
Finished with: 1175 of 1657
Finished with: 1176 of 1657
Finished with: 1177 of 1657
Finished with: 1178 of 1657
Finished with: 1179 of 1657
Finished with: 1180 of 1657
Skipping: 1181 due to <20 min or not classic
Finished with: 1182 of 1657
Finished with: 1183 of 1657
Finished with: 1184 of 1657
Finished with: 1185 of 1657
Skipping: 1186 due to <20 min or not classic
Skipping: 1187 due to <20 min or not classic
Finished with: 1188 of 1657
Finished with: 1189 of 1657
Finished with: 1190 of 1657
Finished with: 1191 of 1657
Skipping: 1192 due to <20 min 

Finished with: 1406 of 1657
Finished with: 1407 of 1657
Skipping: 1408 due to <20 min or not classic
Finished with: 1409 of 1657
Finished with: 1410 of 1657
Finished with: 1411 of 1657
Skipping: 1412 due to <20 min or not classic
Finished with: 1413 of 1657
Finished with: 1414 of 1657
Finished with: 1415 of 1657
Finished with: 1416 of 1657
Skipping: 1417 due to <20 min or not classic
Skipping: 1418 due to <20 min or not classic
Finished with: 1419 of 1657
Finished with: 1420 of 1657
Finished with: 1421 of 1657
Finished with: 1422 of 1657
Finished with: 1423 of 1657
Finished with: 1424 of 1657
Finished with: 1425 of 1657
Finished with: 1426 of 1657
Finished with: 1427 of 1657
Skipping: 1428 due to <20 min or not classic
Skipping: 1429 due to <20 min or not classic
Finished with: 1430 of 1657
Finished with: 1431 of 1657
Finished with: 1432 of 1657
Finished with: 1433 of 1657
Finished with: 1434 of 1657
Finished with: 1435 of 1657
Finished with: 1436 of 1657
Finished with: 1437 of 1657
Fi

### Save the Prepared DF as a .csv

In [8]:
#This will save the prepared dataframe as .csv using the username format. 
#This will prevent confusion and allow us to easily load and concatenate 
#all of our data into one dataframe later.

df.to_csv(f'prepared_data_{username}', index = False)

### Done!

In [9]:
df

,assistsplayer_1,assistsplayer_10,assistsplayer_2,assistsplayer_3,assistsplayer_4,assistsplayer_5,assistsplayer_6,assistsplayer_7,assistsplayer_8,assistsplayer_9,...,team_totalGold_100,team_totalGold_200,team_trueDamageDoneToChampions_100,team_trueDamageDoneToChampions_200,team_ward_player_100,team_ward_player_200,team_assistsplayer_100,team_assistsplayer_200,team_xp_100,team_xp_200
0,2.0,2.0,8.0,1.0,6.0,9.0,3.0,3.0,1.0,2.0,...,44611,33281,4584,2667,199,144,26,11,46733,39425
1,3.0,4.0,5.0,1.0,1.0,2.0,2.0,3.0,4.0,2.0,...,34073,34423,1404,1704,137,81,12,15,38296,40562
2,2.0,6.0,1.0,1.0,2.0,3.0,1.0,7.0,5.0,2.0,...,37287,33537,782,1566,70,71,9,21,40381,41764
3,3.0,6.0,3.0,7.0,4.0,9.0,1.0,1.0,0.0,3.0,...,42884,30572,2489,1395,46,83,26,11,44227,36740
4,2.0,1.0,8.0,3.0,6.0,14.0,0.0,2.0,2.0,0.0,...,40223,33174,1507,2362,118,45,33,5,45743,40135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1086,1.0,4.0,1.0,2.0,3.0,8.0,4.0,3.0,6.0,5.0,...,35738,36509,656,1356,46,42,15,22,40081,44132
1087,0.0,8.0,3.0,4.0,6.0,8.0,0.0,3.0,1.0,7.0,...,37219,36452,1990,2273,121,86,21,19,40130,38655
1088,2.0,9.0,4.0,1.0,3.0,4.0,1.0,2.0,1.0,3.0,...,33964,39725,1762,1444,53,57,14,16,38826,43382
1089,4.0,5.0,3.0,5.0,2.0,10.0,2.0,1.0,2.0,6.0,...,38397,36182,2333,4000,44,44,24,16,40614,39679
